In [1]:
from divisiveclustering.datautils import DataUtils
from divisiveclustering.coresetsUtils import Coreset, normalize_np ,gen_coreset_graph
from divisiveclustering.vqe_utils import kernel_two_local, create_Hamiltonian_for_K2, get_Hamil_variables
from divisiveclustering.quantumutils import get_probs_table
import cudaq
import numpy as np
import warnings
from divisiveclustering.coresetsUtils import gen_coreset_graph
warnings.filterwarnings("ignore")
import pandas as pd
from divisiveclustering.helpers import add_children_to_hc

In [2]:
number_of_qubits = 5
layer_count = 1
parameter_count = 4 * layer_count * number_of_qubits

In [3]:
data_utils = DataUtils('../../data')

try:
    raw_data = data_utils.load_dataset()
except:
    raw_data = data_utils.create_dataset(n_samples = 1000)

Data loaded from ../../data/dataset.pickle


In [4]:
coreset_vectors = data_utils.load_dataset("5_coreset_vectors.npy")
coreset_weights = data_utils.load_dataset("5_coreset_weights.npy") 

In [5]:

coreset_points, G, H, weight_matrix, weights = gen_coreset_graph(
    coreset_vectors, coreset_weights, metric="dot"
)

# TODO: convert to a function
df = pd.DataFrame(coreset_vectors, columns=list("XY"))

df["Name"] = [chr(i + 65) for i in df.index]

index_vals = [i for i in range(len(coreset_weights))]
hc = [index_vals]

i = 0
single_clusters = 0
max_iterations = 100
max_shots = 1000


In [13]:

while single_clusters < len(index_vals):
    print(f"Iteration: {i}")
    if i > 0:
        hc = data_utils.load_object("VQE","hc")
    if len(hc[i]) == 1:
        single_clusters += 1
        i += 1
    else:
        index_values_to_evaluate = hc[i]
        df_to_evaluate = df.iloc[index_values_to_evaluate]
        df_to_evaluate = df_to_evaluate.drop(columns=["Name"])
        G, weights, qubits = get_Hamil_variables(
            coreset_vectors, coreset_weights, index_values_to_evaluate, df_to_evaluate
        )
        print(f"Qubits: {qubits}")
        hamiltonian = create_Hamiltonian_for_K2(G, number_of_qubits, weights)
        
        parameter_count = 4 * layer_count * qubits
        
        optimizer = cudaq.optimizers.COBYLA()
        optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)
        optimizer.max_iterations = max_iterations
        
        optimal_expectation, optimal_parameters = cudaq.vqe(
            kernel=kernel_two_local(qubits, layer_count),
            spin_operator=hamiltonian,
            optimizer=optimizer,
            parameter_count=parameter_count,
            shots = max_shots)
        
        counts = cudaq.sample(kernel_two_local(qubits, layer_count), optimal_parameters, shots_count = max_shots)
        probs_table = get_probs_table(counts, sort_values= True)
        best_bitstring = probs_table.iloc[0][0]
        # TODO: remove the 000 and 111
        
        df_to_evaluate["clusters"] = [int(i) for i in best_bitstring]
        hc = add_children_to_hc(df_to_evaluate, hc)
        
        data_utils.save_object("VQE", hc)
        # save object
        
        i += 1

        
        
        
        

Iteration: 242
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 243
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 244
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
2
<H> = -0.392310
<H> = -0.392227
<H> = -0.392382
<H> = -0.392668
<H> = -0.392673
<H> = -0.392594
<H> = -0.392644
<H> = -0.392596
<H> = -0.392668
<H> = -0.392695
<H> = -0.392689
<H> = -0.392441
<H> = -0.392559
<H> = -0.392658
<H> = -0.392680
<H> = -0.392501
<H> = -0.392701
<H> = -0.355664
<H> = -0.355664
<H> = 0.031779
<H> = -0.355669
<H> = 0.104942
<H> = -0.356082
<H> = -0.354929
<H> = -0.357482
<H> = -0.123682
<H> = -0.356411
<H> = -0.064259
<H> = -0.357781
<H> = -0.260247
<H> = -0.153193
<H> = -0.357781
<H> = -0.012611
<H> = -0.340066
<H> = 0.007223
<H> = -0.371132
<H> = -0.241439
<H> = -0.352930
<H> = -0.148755
<H> = -0.165776
<H> = -0.321208
<H> = -0.37

Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 248
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 249
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
0.392734
<H> = -0.392788
<H> = -0.392789
<H> = -0.392719
<H> = -0.392790
<H> = -0.372856
<H> = -0.372856
<H> = 0.007865
<H> = -0.372735
<H> = 0.009485
<H> = -0.372735
<H> = -0.373543
<H> = -0.377892
<H> = -0.010120
<H> = -0.371587
<H> = -0.017449
<H> = -0.377795
<H> = -0.245424
<H> = -0.090633
<H> = -0.274502
<H> = -0.377892
<H> = -0.270601
<H> = -0.377901
<H> = -0.325466
<H> = -0.377899
<H> = -0.317360
<H> = -0.377869
<H> = -0.249717
<H> = -0.378127
<H> = -0.328729
<H> = -0.377289
<H> = -0.278923
<H> = -0.379523
<H> = -0.291154
<H> = -0.379655
<H> = -0.348324
<H> = -0.379442
<H> = -0.236141
<H> = -0.379733
<H> = -0.324298
<H> = -0.379701
<H> = -0.373203
<H> = -0.379620
<H> = -0.350343
<H> = -0.376791
<H> = -0.356062
<H> = -0.380077
<

Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 253
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 254
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
<H> = -0.116010
<H> = -0.347777
<H> = -0.142287
<H> = -0.347801
<H> = -0.276801
<H> = -0.212537
<H> = -0.315392
<H> = -0.285268
<H> = -0.348423
<H> = -0.306952
<H> = -0.348495
<H> = -0.341508
<H> = -0.348611
<H> = -0.367865
<H> = -0.376387
<H> = -0.378450
<H> = -0.383631
<H> = -0.383574
<H> = -0.364973
<H> = -0.383572
<H> = -0.359174
<H> = -0.383721
<H> = -0.378794
<H> = -0.386649
<H> = -0.368407
<H> = -0.386562
<H> = -0.385458
<H> = -0.388244
<H> = -0.384074
<H> = -0.387657
<H> = -0.379787
<H> = -0.387604
<H> = -0.379732
<H> = -0.387964
<H> = -0.372816
<H> = -0.382088
<H> = -0.384617
<H> = -0.388387
<H> = -0.386646
<H> = -0.388479
<H> = -0.391411
<H> = -0.391274
<H> = -0.391564
<H> = -0.392081
<H> = -0.392508
<H> = -0.392520
<H> = -0

 -0.208153
<H> = -0.375667
<H> = -0.266969
<H> = -0.375699
<H> = -0.278745
<H> = -0.375786
<H> =Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 259
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3
 -0.360766
<H> = -0.375929
<H> = -0.334932
<H> = -0.372650
<H> = -0.362207
<H> = -0.376514
<H> = -0.280512
<H> = -0.368267
<H> = -0.278544
<H> = -0.380326
<H> = -0.278685
<H> = -0.383994
<H> = -0.372506
<H> = -0.383662
<H> = -0.375591
<H> = -0.381985
<H> = -0.281989
<H> = -0.384718
<H> = -0.281885
<H> = -0.384686
<H> = -0.311487
<H> = -0.384852
<H> = -0.381979
<H> = -0.384952
<H> = -0.373417
<H> = -0.391704
<H> = -0.325248
<H> = -0.390749
<H> = -0.320360
<H> = -0.330868
<H> = -0.361777
<H> = -0.390494
<H> = -0.369846
<H> = -0.391147
<H> = -0.377922
<H> = -0.391713
<H> = -0.388787
<H> = -0.391636
<H> = -0.372288
<H> = -0.386762
<H> = -0.391697
<H> = -0.386814
<H> = -0.391741
<H> = -0.387671
<H> = -0.391702
<H> = -0.383735
<H> = -0.391622
<H> = -0.388237
<H> = -0.3910

12
<H> = -0.321789
<H> = -0.383076
<H> = -0.245923
<H> = -0.387282
<H> = -0.278098
<H> = -0.385842
<H> = -0.348972
<H> = -0.385290
<H> = -0.368328
<H> = -0.388473
<H> = -0.285563
<H> = -0.387297
<H> = -0.356696
<H> = -0.387522
<H> = -0.360941
<H> = -0.359794
<H> = -0.388688
<H> = -0.379179
<H> = -0.386728
<H> = -0.376361
<H> = -0.387695
<H> = -0.377868
<H> = -0.385743
<H> = -0.388717
<H> = -0.382907
<H> = -0.388707
<H> = -0.381567
<H> = -0.388168
<H> = -0.387148
<H> = -0.388720
<H> = -0.387100
<H> = -0.390965
<H> = -0.392476
<H> = -0.391618
<H> = -0.392474
<H> = -0.392246
<H> = -0.392459
<H> = -0.391499
<H> = -0.392506
<H> = -0.392701
<H> = -0.392616
<H> = -0.392262
<H> = -0.392683
<H> = -0.392541
<H> = -0.392717
<H> = -0.392216
<H> = -0.392715
<H> = -0.391565
<H> = -0.392682
<H> = -0.391334
<H> = -0.392424
<H> = -0.392671
<H> = -0.392621
<H> = -0.392728
<H> = -0.392671
<H> = -0.392732
<H> = -0.392730
<H> = -0.392828
<H> = -0.392794
<H> = -0.392819
<H> = -0.392794
<H> = -0.392832
<H> =

Data saved in ../../data/data/VQE/VQE_data.pickle
Iteration: 269
Data loaded from ../../data/data/VQE/VQE_data.pickle
Qubits: 3


KeyboardInterrupt: 

In [15]:
probs_table

,bitstring,probability
0,000,0.994
3,111,0.004
1,001,0.001
2,101,0.001


In [ ]:
i